In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('input_output.csv')

In [3]:
df.head(1)

,input_text,target_text
0,Excursion details -1st Stage Separator train B...,- If feasible line up 34' Bypass in line wh...


In [4]:
pd.set_option('display.max_colwidth',None)

In [5]:
df.head(1)

,input_text,target_text
0,Excursion details -1st Stage Separator train B oil level went more than high high limit 2 to 3 time i.e. more than 89.5% after startup.\n\nAnalysis - Level control valve was 100% open working as per level control philosphy but more flow can be achieved through bypass\n,- If feasible line up 34' Bypass in line whenevr level build up suddenly in any of two 1st stage separator train A or B


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_df, val_df = train_test_split(df, test_size=0.1,random_state=1)

In [8]:
train_df.index

Index([41, 19, 35, 34, 33, 43, 32, 23, 17, 31, 29, 36, 40,  4, 24, 14, 10, 39,
       26, 27, 38, 20, 18, 25,  6, 28, 13,  7, 42,  1, 16,  0, 15,  5, 11,  9,
        8, 12, 37],
      dtype='int64')

In [9]:
val_df.index

Index([3, 2, 30, 21, 22], dtype='int64')

In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader


C:\Users\Rajeev\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
class ParagraphDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer
        self.input_texts = data['input_text'].tolist()
        self.target_texts = data['target_text'].tolist()
        self.max_length = max_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        source = self.input_texts[idx]
        target = self.target_texts[idx]

        source_tokenized = self.tokenizer(source, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        target_tokenized = self.tokenizer(target, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")

        return {"input_ids": source_tokenized.input_ids.squeeze(), "attention_mask": source_tokenized.attention_mask.squeeze(), "labels": target_tokenized.input_ids.squeeze()}


In [12]:
# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# Prepare the datasets
train_dataset = ParagraphDataset(tokenizer, train_df)
val_dataset = ParagraphDataset(tokenizer, val_df)


In [20]:
train_dataset.input_texts

['Excursion details MP compressor Discharge and Seperator pressure fluctuation\r\n\r\nAnalysis  It has been observed that MP compressor discharge pressure is fluctuating a lot in the range of 46 bar to 53 bar (LSLL 40Bar) ,The earlier operating range was 49-52Bar .This fluctuation range is observed after 29th March. from both first stage separator as well which might be because of sluggish flow\r\n\r',
 'Excursion details - HP 2 suction scrubber VZ3106A Level transmitter 5LI310619A &5LI310620A not working\r\n\r\nAnalysis  .HP 2 suction scrubber VZ3106A both level transmitter 5LI310620A and 5LI310619A. has been observed to be continuously on  ve side. This has been particularly observed after the last startup, Level Build up and carryover to The Compressor is highly detrimental to the Machine Healthiness\r\n\r',
 'Excursion details It has been observed that Fuel gas filter VF7001A/B PDIT USN_5PDI700142.PV is faulty\r\n\r\nAnalysis  1)It has  been observed that fuel gas filter DP USN_5PD

In [14]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)


In [15]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

C:\Users\Rajeev\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
# Train the model
trainer.train()

Step,Training Loss
10,19.614900
20,18.318000
30,18.764200


TrainOutput(global_step=30, training_loss=18.89904276529948, metrics={'train_runtime': 249.2548, 'train_samples_per_second': 0.469, 'train_steps_per_second': 0.12, 'total_flos': 15834990772224.0, 'train_loss': 18.89904276529948, 'epoch': 3.0})

In [18]:
# Prediction
input_text = "Excursion details -1st stage separator pressure went high upto 15 bar though set point was 7 bar. Analysis - Vessel pressure went more than two time of set pressure and flare valve opened till 100%"
input_tokenized = tokenizer.encode(input_text, return_tensors="pt")

output_tokens = model.generate(input_tokenized, max_length=512, num_beams=5, early_stopping=True)
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(output_text)


Excursion Details - -1st stage separator pressure went high up to 15 bar though set point was 7 bar. Analysis - Vessel pressure went high up to 15 bar though set point was 7 bar.
